In [81]:
"""
Tshikana Rasehlomi
e-mail: rasehlomi@gmail.com
DS_CODE CHALLENGE
QUESTION 4: Classification challenge
10-07-2022
"""
# Load required modules
#============================================================================================================================
import pandas as pd
import numpy as np
from pycaret.datasets import get_data
from sklearn.preprocessing import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score, mean_squared_error, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

In [82]:
# read and load the sr_hex.csv data
#===============================================================================================================
sr_hex_df = pd.read_csv('https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/sr_hex.csv.gz', compression='gzip', header=0, sep=',', quotechar='\"')
# convert timestamp to datetime
#================================================================================================================
sr_hex_df['creation_timestamp'] = pd.to_datetime(df['creation_timestamp'])
# set the index using creation_timestamp
#===============================================================================================================
sr_hex_df.set_index("creation_timestamp", inplace=True )

In [83]:
# extracting input dataframe to be used in a model
#===============================================================================================================
sr_hex_df_mod = df.drop(['reference_number', 'completion_timestamp', 'notification_number',
       'directorate', 'department', 'branch', 'section', 'latitude','cause_code_group',  'official_suburb',
       'longitude', 'h3_level8_index'], axis=1)
sr_hex_df_mod.head()

,code_group,code,cause_code
creation_timestamp,,,
2020-10-07 06:55:18+02:00,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Wear and tear
2020-07-09 16:08:13+02:00,TD Customer complaint groups,Manhole Cover/Gully Grid,Vandalism
2020-10-27 10:21:59+02:00,TD Customer complaint groups,Manhole Cover/Gully Grid,Vandalism
2020-03-19 06:36:06+02:00,TD Customer complaint groups,Paint Markings Lines&Signs,Wear and tear
2020-08-25 09:48:42+02:00,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Surfacing failure


In [84]:
# replace missing values with 0
#======================================
sr_hex_df_mod = sr_hex_df_mod.fillna(0)

In [85]:
# Convert categorical (text) fields to numbers
#===================================================================================================
number = LabelEncoder()
sr_hex_df_mod['code'] = number.fit_transform(sr_hex_df_mod['code'].astype('str'))
sr_hex_df_mod['cause_code'] = number.fit_transform(sr_hex_df_mod['cause_code'].astype('str'))
sr_hex_df_mod['code_group'] = number.fit_transform(sr_hex_df_mod['code_group'].astype('str'))
print(sr_hex_df_mod)

                           code_group  code  cause_code
creation_timestamp                                     
2020-10-07 06:55:18+02:00          25   297         180
2020-07-09 16:08:13+02:00          25   220         167
2020-10-27 10:21:59+02:00          25   220         167
2020-03-19 06:36:06+02:00          25   280         180
2020-08-25 09:48:42+02:00          25   297         149
...                               ...   ...         ...
2020-12-31 23:49:38+02:00          31   203           0
2020-12-31 23:31:11+02:00          19   367          47
2020-12-31 23:58:21+02:00          31    41           0
2020-12-31 23:41:57+02:00          35   241           0
2020-12-31 23:54:09+02:00          31   205           0

[941634 rows x 3 columns]


In [86]:
# prepare data for training
#=============================================================================
#df_model = df_model.fillna(0)
X = sr_hex_df_mod.iloc[:, 0:2].values
y = sr_hex_df_mod.iloc[:, 2].values

# Split the dataset into Train set and Test set
#==============================================================================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [87]:
# Perform feature scaling
#===============================================================================
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [88]:
# compute the RandomForest classifier
#==================================================================================
classifier = RandomForestClassifier(n_estimators = 70, random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

# evaluate the created algorithm
#===================================================================================
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[202657      0      0 ...      0      0      0]
 [     3      0      0 ...      0      0      0]
 [    16      0      0 ...      0      0      0]
 ...
 [   343      0      0 ...      0      0      0]
 [     9      0      0 ...      0      0      0]
 [     1      0      0 ...      0      0      0]]
              precision    recall  f1-score   support

           0       0.86      1.00      0.93    202657
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00        16
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00       664
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00        

0.8608719292805288


C:\Users\TRasehlomi.TRASEHLOMI-NB\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\TRasehlomi.TRASEHLOMI-NB\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\TRasehlomi.TRASEHLOMI-NB\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)